In [1]:
# import the necessary libraries

import pandas as pd
import numpy as np
from geopy.distance import distance, geodesic
from sklearn.neighbors import BallTree

# import the dataset

df = pd.read_csv("C:/Users/agust/OneDrive/Escritorio/Agus/Agus/Proyectos - Portfolio/NY/AB_NYC_2019.csv")

# convert last_review column into a datetime column

df.last_review = pd.to_datetime(df['last_review'], errors='coerce')

# create a new column with the grouped amount of reviews
df['number_of_reviews-grouped'] = np.where(df['number_of_reviews'] == 0, 'No Reviews',
                        np.where(df['number_of_reviews'] <= 5, '1-5',
                                 np.where(df['number_of_reviews'] <= 10, '6-10',
                                          np.where(df['number_of_reviews'] <= 20, '11-20',
                                              np.where(df['number_of_reviews'] <= 50, '21-50',
                                                   np.where(df['number_of_reviews'] <= 100, '51-100',
                                                            'More than 100'))))))

# create a new column with the prices grouped by categories
"""
script to obtain the limits of each of the 5 bins to group prices which I am using in this step

percentiles = np.linspace(0, 100, 6)

# Obtain starting and ending values
valores_limite = np.percentile(df.price, percentiles)

"""

df['price-grouped'] = np.where(df['price'] <= 60, 'Less than $60',
                        np.where(df['price'] <= 90, '$60-$90',
                                 np.where(df['price'] <= 130, '$90-$130',
                                          np.where(df['price'] <= 200, '$130-$200',
                                                            'More than $200'))))

# create a new column with the price grouped
df['price_group'] = pd.cut(df['price'], bins=[0, 60, 130, 200, np.inf], labels=['cheap', 'regular', 'expensive', 'very expensive'])




# dataframe of NY 12 most iconic attractions

nyc_attractions = [
    {"attraction": "Statue of Liberty", "latitude": 40.689247, "longitude": -74.044502},
    {"attraction": "Empire State Building", "latitude": 40.748817, "longitude": -73.985428},
    {"attraction": "Central Park", "latitude": 40.782865, "longitude": -73.965355},
    {"attraction": "Times Square", "latitude": 40.758895, "longitude": -73.985131},
    {"attraction": "Brooklyn Bridge", "latitude": 40.706086, "longitude": -73.996864},
    {"attraction": "One World Trade Center", "latitude": 40.7127, "longitude": -74.0134},
    {"attraction": "Rockefeller Center", "latitude": 40.7587, "longitude": -73.9787},
    {"attraction": "The Metropolitan Museum of Art", "latitude": 40.7794, "longitude": -73.9632},
    {"attraction": "The High Line", "latitude": 40.7479, "longitude": -74.0048},
    {"attraction": "Grand Central Terminal", "latitude": 40.7527, "longitude": -73.9772},
    {"attraction": "The Brooklyn Museum", "latitude": 40.6712, "longitude": -73.9632},
    {"attraction": "St. Patrick's Cathedral", "latitude": 40.7586, "longitude": -73.9762}
]

# create a Pandas DataFrame from the list
attractions = pd.DataFrame(nyc_attractions)

# Create a new column in the apartments dataset for each of the attractions and the distance from the apartment to the attraction
for i in nyc_attractions:
    # Calcular la distancia utilizando la fórmula de Haversine
    df[i.get("attraction")] = df.apply(lambda row: distance((row['latitude'], row['longitude']), (i.get("latitude"),i.get("longitude"))).m, axis=1)
    
# import a dataset with all NY subway stations and their latitude and longitude    
subwayStations = pd.read_csv("C:/Users/agust/OneDrive/Escritorio/Agus/Agus/Proyectos - Portfolio/NY/NYC_Transit_Subway_Entrance_And_Exit_Data.csv",usecols=['Station Latitude', 'Station Longitude'])    
subwayStations.drop_duplicates(subset=['Station Latitude', 'Station Longitude'], inplace=True)


# create a new ball tree algoritm using subway stations latitudes and longitudes to get the distance to the closest subway station
subway_tree = BallTree(np.radians(subwayStations[['Station Latitude', 'Station Longitude']].values), leaf_size=40, metric='haversine')


distances, indices = subway_tree.query(np.radians(df[['latitude', 'longitude']].values), k=1)

df['subway_station_latitude'] = subwayStations.iloc[indices.flatten()]['Station Latitude'].values
df['subway_station_longitude'] = subwayStations.iloc[indices.flatten()]['Station Longitude'].values

df['subway_station_distance'] = df.apply(lambda row: distance((row['latitude'], row['longitude']), (row['subway_station_latitude'], row['subway_station_longitude'])).m, axis=1)

df.drop(columns=['subway_station_latitude', 'subway_station_longitude'], inplace=True)

# create a new column with the distance to the subway station grouped

df['StationDistance - Grouped'] = np.where(df['subway_station_distance'] <= 100, 'Less than 100m',
                        np.where(df['subway_station_distance'] <= 200, '100m-200m',
                                 np.where(df['subway_station_distance'] <= 300, '200m-300m',
                                          np.where(df['subway_station_distance'] <= 400, '300m-400m',
                                              np.where(df['subway_station_distance'] <= 500, '400m-500m',
                                                            'More than 500m')))))

# remove the rows that are outliers in the top and remove the rows with price equal to 0
# calculate the quartiles and IQR
Q1 = df['price'].quantile(0.01)
Q3 = df['price'].quantile(0.95)
IQR = Q3 - Q1

# calculate the lower and upper limits
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# identify the outliers
df = df[~ ((df['price'] > upper_limit) | (df['price'] ==0))]